## Combinatorial Optimization

#### Assignment IV

***Heuristics for the Knapsack Problem - Solving Multiple Instances***

**Instance:** C30I20

Guilherme Cadori

09/07/2023


In [2]:
# Importing required libraries
import numpy as np


#### 1. Read the instance

In [3]:
# Reading the target file
# GC: the code template below was provided in class and was adapted by GC (2023-07-09)

filename = "C:\gccadori\Lab4_CO\Knap_C30I20.dat"  # Substitua pelo nome do arquivo real

# Variáveis para armazenar os dados lidos
linha = None
coluna = None
cost = None
rhs = None
matrix = None

# Função auxiliar para converter uma linha de valores em uma lista de floats
def parse_values(line):
    return [float(value) for value in line.strip().split()]

# Ler o arquivo e extrair os dados
with open(filename, "r") as file:
    lines = file.readlines()

# Ler o escalar
linha = int(lines[0].strip())
coluna = int(lines[2].strip())

# Ler o vetor
cost = np.array(parse_values(lines[5]))

# Ler a matriz
end = 7 + linha
matrix_lines = [line.strip() for line in lines[7:end]]
matrix = np.array([parse_values(line) for line in matrix_lines],dtype=object)

# Ler vetor de RHS
rhs = np.array(parse_values(lines[end+1]))

# Imprimir os dados lidos
print("\n linhas:", linha)
print("\n colunas:", coluna)
print("\n cost:", cost)
print("\n rhs:", rhs)
print("\n Matrix:")
for row in matrix:
    print(row)



 linhas: 30

 colunas: 20

 cost: [0.30876 0.27366 0.40452 0.78949 0.99419 0.26734 0.13833 0.91129 0.10879
 0.45288 0.6373  0.08602 0.81526 0.05092 0.77633 0.85511 0.87917 0.1723
 0.89771 0.84468]

 rhs: [162.68004 178.80225 199.18652 218.22931 193.79358 179.28003 215.02693
 218.88757 164.77    192.88625 208.58598 173.80133 182.70836 240.55583
 188.87265 186.98349 138.40446 186.41516 237.99693 172.38055 199.76094
 165.92651 208.38326 196.09937 191.49927 211.63223 180.99176 171.4373
 216.65773 169.84282]

 Matrix:
[12.73264 22.44011 0.93729 2.96613 1.81641 3.38822 20.89636 5.17353
 1.57997 4.60206 21.84674 3.16493 17.98423 10.98489 22.96316 16.83319
 15.33118 21.17208 14.22625 0.62673]
[0.69962 8.46768 16.29891 10.78457 6.19274 6.38173 7.68375 15.86962
 20.69212 22.39649 17.86554 16.11791 18.6629 17.34636 15.3464 1.96366
 3.21747 1.05363 8.33088 17.15013]
[14.60403 24.98539 4.41362 4.70763 21.13298 6.98283 10.38871 3.0565
 20.52219 16.44674 20.32751 19.47576 4.08472 1.86515 22.56647 24

#### 2.	Construct an initial feasible solution

In [4]:
# Renaming variables for testing purposes
c = cost

A = matrix

b = rhs


In [5]:
# Providing an initial feasible solution

def randomStart(nVars, A, b, c):
    # Creating variable names x1, ..., x_nVars
    variable_names = ['x{}'.format(i) for i in range(1, nVars+1)]

    while True:
        np.random.seed(12345)
        x = np.random.randint(2, size=nVars)
        
        if np.all(A @ x <= b):
            FO = x @ c
            print('The initial feasible solution is:')   
            print('Objective function value:', round(FO, 2))
            return


In [6]:
# Testing the function
randomStart(nVars = coluna, A = A, b = b, c = c)


The initial feasible solution is:
Objective function value: 5.34


#### 3.	Program the heuristic you proposed in the previous assingment

In [7]:
# Implementing the heuristic algorithm proposed in Lab III

def naiveKnapsack(nVars, items_c, capacity_b, A):
    # Sort the items in descending order of value
    sorted_items = sorted(range(nVars), key=lambda i: items_c[i], reverse=True)

    # Initialize variables
    selected_items = np.zeros(nVars, dtype=int)
    total_weight = np.zeros_like(capacity_b, dtype=items_c.dtype)
    total_value = 0

    # Greedy selection process
    for idx in sorted_items:
        weight = A[:, idx]
        if np.all(total_weight + weight <= capacity_b):
            # Add the item to the knapsack
            selected_items[idx] = 1
            total_weight += weight.astype(total_weight.dtype)  # Ensure matching data type
            total_value += items_c[idx]

    # Print the selected items and total value
    print('The initial feasible solution is:')
    print('Objective function value:', round(total_value, 2))

    # Return the complete list of items
    return


In [8]:
# Testing the function
naiveKnapsack(nVars = coluna, items_c = c, capacity_b = b , A = A)


The initial feasible solution is:
Objective function value: 9.53


#### 4.	Program a Local Search 

In [9]:
# Local Search metaheuristic implementation
def LocalSearch(nVars, A, b, c, nStarts=1, max_iterations=1000):
    import numpy as np

    # Creating variable names
    variable_names = ['x{}'.format(i) for i in range(1, nVars + 1)]

    best_solution = None
    best_value = -np.inf

    # Escaping Local Optima - Multiple Starting Solutions
    for n in range(1, nStarts + 1):
        # Producing an initial feasible solution
        while True:
            x = np.random.randint(2, size=nVars)
            if np.all(A @ x <= b):
                FO = x @ c
                break  # Exit the loop once an initial feasible solution is found

        # Setting initial search parameters
        s = FO
        s_new = 0
        n_iterations = 0
        best_iteration = 0

        # Termination criteria
        while n_iterations - best_iteration < max_iterations:
            n_iterations += 1

            # Neighbor Selection: Swap Two - First Improvement Strategy
            def generate_neighbor_solution(x):
                x_new = x.copy()
                idx1, idx2 = np.random.choice(len(x), size=2, replace=False)
                x_new[idx1], x_new[idx2] = x_new[idx2], x_new[idx1]
                return x_new

            # Generating neighbor solution
            x_new = generate_neighbor_solution(x)

            # Evaluate solution
            if np.all(A @ x_new <= b):
                FO_new = x_new @ c
            else:
                FO_new = -np.inf

            # Update the current solution
            if FO_new > s_new:
                x = x_new
                s_new = FO_new
                best_iteration = n_iterations

                # Check if the current solution is the best found so far
                if s_new > best_value:
                    best_solution = x.copy()
                    best_value = s_new

    # Print the best solution found
    print('The best solution found:')
    print('Objective function value:', round(best_value, 2))

    return


#### 5.	Running the heuristic for the given instance

In [10]:
import time

# Record starting time
start_time = time.time()

# Running the proposed heuristic for this instance
LocalSearch(nVars = coluna, A = A, b = b, c = c, 
            nStarts = 100,
            max_iterations = 500)

# Record ending time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = round(end_time - start_time, 2)

# Print the elapsed time
print("Elapsed time:", elapsed_time, "seconds")


The best solution found:
Objective function value: 9.53
Elapsed time: 4.62 seconds


**Test Run Results**

The best solution found:

- Objective function value: 9.53
- Elapsed time: 4.62 seconds


### End